<a href="https://colab.research.google.com/github/Jbadro/LLM-creation/blob/main/TechX2024_nanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget http://gir.fyi/techx/input.txt

--2024-05-15 19:01:45--  http://gir.fyi/techx/input.txt
Resolving gir.fyi (gir.fyi)... 216.239.32.21, 216.239.34.21, 216.239.38.21, ...
Connecting to gir.fyi (gir.fyi)|216.239.32.21|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt [following]
--2024-05-15 19:01:46--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-05-15 19:01:46 (52.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
  raw_training_data = f.read()

print(len(raw_training_data))

1115394


In [5]:
len(set(raw_training_data))

65

In [38]:
''.join(sorted(set(raw_training_data)))

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [6]:
from collections import defaultdict

freqs = defaultdict(int)

for letter in raw_training_data:
  freqs[letter] += 1

freqs

defaultdict(int,
            {'F': 1797,
             'i': 45537,
             'r': 48889,
             's': 49696,
             't': 67009,
             ' ': 169892,
             'C': 3820,
             'z': 356,
             'e': 94611,
             'n': 48529,
             ':': 10316,
             '\n': 40000,
             'B': 2761,
             'f': 15770,
             'o': 65798,
             'w': 17585,
             'p': 10808,
             'c': 15623,
             'd': 31358,
             'a': 55507,
             'y': 20448,
             'u': 26584,
             'h': 51310,
             ',': 19846,
             'm': 22243,
             'k': 7088,
             '.': 7885,
             'A': 7819,
             'l': 33339,
             'S': 4523,
             'Y': 1718,
             'v': 7793,
             '?': 2462,
             'R': 4869,
             'M': 2840,
             'W': 3530,
             "'": 6187,
             'L': 3876,
             'I': 11832,
             'N': 5079,

In [8]:
import random

SAMPLE_SIZE = 500
training_data_size = len(raw_training_data)
start = random.randrange(0, training_data_size - SAMPLE_SIZE)
print(raw_training_data[start:start+SAMPLE_SIZE])

e heavy accent of thy moving tongue
And in compassion weep the fire out;
And some will mourn in ashes, some coal-black,
For the deposing of a rightful king.

NORTHUMBERLAND:
My lord, the mind of Bolingbroke is changed:
You must to Pomfret, not unto the Tower.
And, madam, there is order ta'en for you;
With all swift speed you must away to France.

KING RICHARD II:
Northumberland, thou ladder wherewithal
The mounting Bolingbroke ascends my throne,
The time shall not be many hours of age
More than 


In [13]:
tokens = ''.join(sorted(set(raw_training_data)))
token_to_number = {t: i for i, t in enumerate(tokens)}

tokens

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [11]:
number_to_token = {i: t for i, t in enumerate(tokens)}

In [14]:
def encode(s):
  return_value = []
  for character in s:
    return_value.append(token_to_number[character])
  return return_value

encode("Tech Exchange")

[32, 43, 41, 46, 1, 17, 62, 41, 46, 39, 52, 45, 43]

In [16]:
def decode(nums):
  return ''.join([number_to_token[t_id] for t_id in nums])

decode("s")


<function __main__.decode(nums)>

In [21]:
import torch

data = torch.tensor(encode(raw_training_data), dtype=torch.long)

In [25]:
holdout_size = int(0.1 * len(data))
holdout_size

111539

In [26]:
test_data = data[:holdout_size]
training_data = data[holdout_size:]
len(test_data), len(training_data)

(111539, 1003855)

In [27]:
BLOCK_SIZE = 8
training_data[:BLOCK_SIZE+1]

tensor([43, 58,  6,  1, 25, 39, 56, 41, 47])

In [42]:
start_from = random.randint(0, training_data_size-BLOCK_SIZE)
decode(training_data[start_from:start_from+BLOCK_SIZE].tolist())

''

In [37]:
x = training_data[:BLOCK_SIZE]
y = training_data[1:BLOCK_SIZE+1]
for t in range(BLOCK_SIZE):
  context = x[:t+1]
  target = y[t]
  print(f"in: {context}, out: {target}")

in: tensor([43]), out: 58
in: tensor([43, 58]), out: 6
in: tensor([43, 58,  6]), out: 1
in: tensor([43, 58,  6,  1]), out: 25
in: tensor([43, 58,  6,  1, 25]), out: 39
in: tensor([43, 58,  6,  1, 25, 39]), out: 56
in: tensor([43, 58,  6,  1, 25, 39, 56]), out: 41
in: tensor([43, 58,  6,  1, 25, 39, 56, 41]), out: 47


In [43]:
torch.manual_seed(1337)

BATCH_SIZE = 4


In [51]:
def get_batch(split):
  data = training_data if split == 'train' else test_data
  ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
  print(ix)
  x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
  print(x)
  y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
  print(y)
  return x,y

get_batch('train')

tensor([ 61537,  39924, 120920, 342807])
tensor([[46, 58,  1, 56, 53, 63, 39, 50],
        [43,  6,  1, 54, 43, 56, 44, 47],
        [43, 57, 47, 42, 43, 57,  6,  1],
        [ 1, 58, 53,  1, 40, 43, 46, 53]])
tensor([[58,  1, 56, 53, 63, 39, 50,  6],
        [ 6,  1, 54, 43, 56, 44, 47, 42],
        [57, 47, 42, 43, 57,  6,  1, 46],
        [58, 53,  1, 40, 43, 46, 53, 50]])


(tensor([[46, 58,  1, 56, 53, 63, 39, 50],
         [43,  6,  1, 54, 43, 56, 44, 47],
         [43, 57, 47, 42, 43, 57,  6,  1],
         [ 1, 58, 53,  1, 40, 43, 46, 53]]),
 tensor([[58,  1, 56, 53, 63, 39, 50,  6],
         [ 6,  1, 54, 43, 56, 44, 47, 42],
         [57, 47, 42, 43, 57,  6,  1, 46],
         [58, 53,  1, 40, 43, 46, 53, 50]]))

In [50]:
xb, yb = get_batch('train')
for b in range(BATCH_SIZE):
  for t in range(BLOCK_SIZE):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f'input {context.tolist()}, target {target}')

tensor([683569,  90917,  47439, 700859])
tensor([[53, 56,  1, 21,  1, 51, 59, 57],
        [39, 51, 43,  1, 58, 53,  1, 63],
        [13, 23, 17, 26, 14, 33, 30, 37],
        [46, 39, 52, 45, 43,  2,  1, 31]])
tensor([[56,  1, 21,  1, 51, 59, 57, 58],
        [51, 43,  1, 58, 53,  1, 63, 53],
        [23, 17, 26, 14, 33, 30, 37, 10],
        [39, 52, 45, 43,  2,  1, 31, 59]])
input [53], target 56
input [53, 56], target 1
input [53, 56, 1], target 21
input [53, 56, 1, 21], target 1
input [53, 56, 1, 21, 1], target 51
input [53, 56, 1, 21, 1, 51], target 59
input [53, 56, 1, 21, 1, 51, 59], target 57
input [53, 56, 1, 21, 1, 51, 59, 57], target 58
input [39], target 51
input [39, 51], target 43
input [39, 51, 43], target 1
input [39, 51, 43, 1], target 58
input [39, 51, 43, 1, 58], target 53
input [39, 51, 43, 1, 58, 53], target 1
input [39, 51, 43, 1, 58, 53, 1], target 63
input [39, 51, 43, 1, 58, 53, 1, 63], target 53
input [13], target 23
input [13, 23], target 17
input [13, 23, 17]

In [52]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [54]:
class BigramLanguageModel(nn.Model):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

AttributeError: module 'torch.nn' has no attribute 'Model'